# Vienna District Recommender
In this notebook we create a simple recommender for Vienna's districts. It takes a construed district that represents our preferences and then returns a list of viennas districts, weighted by how well the fit our preferences.

In [1]:
# basic http operations
import requests

# basic computing and data wrangling
import pandas as pd
import numpy as np
import json

# geopandas for advanced geojson handling
import geopandas

# visualizing geo data
import folium

# library to access overpass api in a more convenient way
import overpass

# interactions with the host system 
import os
import time

# pattern matching scala style
# from pampy import match, _

# machine learning
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier

## Load districts borders from OSM
As a first step, we need to download Vienna's districts. We will use them as reference points for downloading data about places later, as well as for data visualization. Open Street Maps is a great source for data like this.

To get data from OSM we can use the overpass api which has a wrapper package for python. We first initialize the API and then we download data about districts borders.

In [2]:
overpass_api = overpass.API(timeout=600)

In [3]:
query = """
area["name"="Wien"]["admin_level" ="4"];
relation["boundary"="administrative"]["admin_level"="9"](area);
(._;>;);
out;
"""

response = overpass_api.get(query, responseformat="json")

Since I found no possibility convert the raw json output of overpass to geojson format, here comes a little trick. We need to use a external programm to achieve this. The programm we use is called 'osmtogeojson' and is written to run on the node.js environment. 

If you want to try the steps below, you need install node.js and then install 'osmtogeojson' by running 'npm install -g osmtogeojson' in your console (npm is the node.js equivalent of pip).

We save the downloaded raw json to the file system und then run osmtogeojson and convert it to a geojson file.


In [4]:
with open("vienna_bounds_raw.json", "w") as file:
    file.write(json.dumps(response))
    
os.system('osmtogeojson vienna_bounds_raw.json > vienna_bounds.geojson')


0

We can now import the generated geojson file using the library geopandas, a pandas derivative that is mainly made for processing geo data like we saved in our geojson. 

Since the geojson file we downloaded still contains data we don't need and which spoils our our visualizations later on, we filter the imported file and save it to the file system again so we can use it with folium later. 


In [5]:
gdf_vienna_bounds = (geopandas.read_file('vienna_bounds.geojson')
                    .loc[:, ['id','name', 'geometry', 'ref']]
                    .query("id.str.startswith('relation')", engine="python")
            )

gdf_vienna_bounds.to_file("vienna_bounds.geojson", driver='GeoJSON')


Lets check our DataFrame containing the data about Viennas districts.

In [6]:
gdf_vienna_bounds.head()


,id,name,geometry,ref
0,relation/1990590,Alsergrund,"POLYGON ((16.3567669 48.2361278, 16.3567424 48...",9
1,relation/1990591,Hietzing,"POLYGON ((16.2584479 48.1972588, 16.2580803 48...",13
2,relation/1990592,Innere Stadt,"POLYGON ((16.3659409 48.1996821, 16.3659676 48...",1
3,relation/1990593,Josefstadt,"POLYGON ((16.3392972 48.2122485, 16.3392509 48...",8
4,relation/1990594,Leopoldstadt,"POLYGON ((16.497085 48.1662737, 16.4984617 48....",2


To make shure we downloaded the correct data, we can visualize with the help of the folium map visualization library.


In [7]:
vienna_map = folium.Map(location=(48.2204,16.3367), zoom_start=11) 

folium.Choropleth(
    geo_data="vienna_bounds.geojson"
).add_to(vienna_map)

                  
vienna_map


## Calculate centroids

As we need a reference point for our foursquare API, we will simply get the centroids of the single polygons representing Viennas districts. As we are using geopandas, which uses shapely under the hood, this is very easy to accomplish.

In [8]:
gdf_vienna = (gdf_vienna_bounds
                .assign(center_lat=lambda x: x['geometry'].centroid.y)
                .assign(center_lon=lambda x: x['geometry'].centroid.x)
             )

Lets display these centroids on the map to check if they look realistic.

In [9]:
for i, district in gdf_vienna.iterrows():
    

    folium.Marker(
        (district['center_lat'], district['center_lon'])
        
    ).add_to(vienna_map)
    
vienna_map

Looks good. Lets continue with retrieving data from foursquare.

## Retrieve trending venues from foursquare

Set foursquare credentials. (Since its a free account I don't really care if they are available on github)

In [10]:
CLIENT_ID = 'TLLSXQMPOKLY0NLU0ZRA4MUW45XBYO5G0P5B5VFW0ZRJMY2U'
CLIENT_SECRET = 'QH2K33NY2RZQPSHRBYGLNMFDYYEPLUTOA44IND2NCDEQVXDD'
VERSION = '20190706'

In [11]:
def get_trending_venues(lat, lon, radius, limit):

    url = "https://api.foursquare.com/v2/venues/explore"
    
    
    
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "v": VERSION,
        "ll": "{lat},{lon}".format(lat=lat, lon=lon),
        "radius": radius,
        "limit": limit,
        "section": 'trending',
        "query": 'food'
    }

    return requests.get(url, params).json()['response']['groups'][0]['items']

We define a function that takes all venues we got for a district and counts them grouped by their primary category. 

In [12]:
def group_venues(venues, d_ref, d_name):
    
    list_venues = (venue['venue']['categories'][0]['name'] for venue in venues) 
    
    df_venues = pd.DataFrame(list_venues, columns=["name"])
    
    
    return (df_venues
             
             .groupby('name')['name']
                .count()
                .to_frame()
             .transpose()
             .reset_index()
             .assign(d_ref=d_ref)
             .assign(d_name=d_name)
             .drop(columns=["index"])
           )

Now we use our functions we defined before to retrieve all venues for all districts and build a list of dataframes. This list can be concatinated then to receive one single dataframe with all the places.

In [13]:
trending_venues = []

for i, district in gdf_vienna.iterrows():
    
    venues = get_trending_venues(district['center_lat'], district['center_lon'], 2000, 110)
    
    trending_venues.append(group_venues(venues, district['ref'], district['name']))

In [14]:
df_trending = (pd.concat(trending_venues, sort=False)
               .fillna(0)
              )

Lets drop all venue categories that don't contain at least 10 entries as they are to specific and it will be impossible in a future implementation to ask the user for his preferences, given so many categories.

In [15]:
df_trending_feature_sel = df_trending.drop(columns=["d_ref", "d_name"]).apply(pd.to_numeric)

is_small = (df_trending_feature_sel.sum() < 10)
col_del = is_small[is_small].index

df_foursquare_places = (df_trending
                          .drop(columns=col_del)
                          .reset_index()
                          .drop(columns=["index", "d_name"])
                          .apply(lambda x: x.astype(int) if x.name == 'd_ref' else x)
                     )

Just a short peek on what we've got so far:

In [16]:
df_foursquare_places.head()

,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Falafel Restaurant,Gastropub,...,Sushi Restaurant,Thai Restaurant,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,d_ref,Bistro,Hot Dog Joint,Steakhouse,Fast Food Restaurant
0,5.0,6,1.0,6.0,1.0,2.0,13.0,2.0,4.0,5.0,...,1.0,2.0,2.0,2.0,2.0,9,0.0,0.0,0.0,0.0
1,1.0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,13,0.0,0.0,0.0,0.0
2,4.0,14,1.0,3.0,1.0,3.0,8.0,1.0,1.0,3.0,...,2.0,2.0,2.0,2.0,0.0,1,1.0,1.0,3.0,0.0
3,5.0,9,0.0,5.0,1.0,3.0,13.0,2.0,1.0,3.0,...,4.0,1.0,1.0,1.0,2.0,8,2.0,1.0,1.0,0.0
4,2.0,3,1.0,7.0,0.0,1.0,7.0,1.0,0.0,2.0,...,0.0,1.0,1.0,0.0,0.0,2,1.0,3.0,0.0,2.0


## Load Open Street Maps amenities
Now we wan't to add some places from open street maps. 

For doing so, we create a function that calls the overpass api for given district and retrieves all parks, supermarket, universities, restaurants, doctors and waste baskets in this district.

In [17]:
def retrieve_amenities(relation_id):
    
    area_id = relation_id + 3600000000 

    query = """
area({area});
way["leisure" = "park"](area);
out center;
 
area({area});
node["shop" = "supermarket"](area);
out; 

area({area});
node["amenity" = "university"](area);
out;

area({area});
node["amenity" = "restaurant"](area);
out;

area({area});
node["amenity" = "doctors"](area);
out;

area({area});
node["amenity" = "waste_basket"](area);
out;
    """.format(area=area_id)
    
    overpass_api.endpoint

    response = overpass_api.get(query, responseformat="json")
    
    return response

We also need a function to group all places by their category and count them. Since category information does not come in the same column, first we need to do some data wrangling. 

In [18]:
def group_places(places, d_name, d_ref, d_area) -> pd.DataFrame:
    

    df_places = pd.io.json.json_normalize(places['elements'])
    
    df_places['category'] = df_places['tags.amenity'].fillna(df_places['tags.leisure'])
    df_places['category'] = df_places['category'].fillna(df_places['tags.shop'])
    
    return  (df_places
                      .groupby('category')['id']
                        .count()
                        .to_frame()
                     .transpose()
                     .reset_index()
                     .assign(index=lambda x: x.index.astype(int),
                         d_area=d_area,
                         d_ref=d_ref,
                         d_name=d_name)
           )
    
    
   


Download all places specified in function **retrieve_amenities**. 
This will take a while (about 1h) since Overpass API has restrictions on multiple access. 

Since download takes so long, we will first check if there is already a chached version on the file system

In [24]:
try: 
    df_osm_places = pd.read_csv('vienna_osm_places.csv')
except FileNotFoundError:
    df_osm_places = load_osm()

The function below is used to download places from overpass and call the grouping function so we get a tidy and grouped dataframe with all districts and all counts of places.

In [23]:
def load_osm():
    places = list()
    for i, row in gdf_vienna.iterrows():

        area = row['geometry'].area


        # Wait 300s if we get any exception and retry. 
        # We don't specify exception type because there are many that lead to a retry
        while True:
            try:
                places_raw = retrieve_amenities(int(row['id'].split('/')[1]))


            except:

                time.sleep(300)
                continue

            break

        places.append(group_places(places_raw, row['name'], row['ref'], area))
       
    # Concat resulting dataframes and return 
    return (pd.concat(places, sort=False)
               .reset_index()
               .drop(columns=['index', 'level_0', 'atm', 'cafe', 'fuel', 'd_name'])
               .rename(columns={
                   'doctors':'osm_doctor',
                   'park':'osm_park',
                   'restaurant':'osm_restaurant',
                   'supermarket':'osm_supermarket',
                   'university':'osm_university',
                   'waste_basket':'osm_waste_basket'
               })
               .apply(lambda x: x.astype(int) if x.name == 'd_ref' else x)
               .fillna(0)                   
              )

Save the data we've gathered to .csv so we don't need to wait for an hour next time we need it.

In [25]:
df_osm_places.to_csv('vienna_osm_places.csv')

## Combine OSM and foursquare data

Simply merge the two dataframes we've created before on the districts reference number.

It makes sense divide the absolute numbers of places by the area of the district because four our user only the density of places he likes is important. Although the area we got from our geojson file does not mirror the real area of the district, but it is sufficient to calculate density of the districts relative to each other. 

In [39]:
df_vienna_features = (df_foursquare_places
                      .merge(
                          right=df_osm_places,
                          how='inner',
                          right_on='d_ref',
                          left_on='d_ref')
                      .set_index('d_ref')
                      .apply(lambda x: x.div(x['d_area']), axis=1)
                      .drop(columns=['d_area', 'Unnamed: 0'])
                     )

df_vienna_features.head()

,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Falafel Restaurant,Gastropub,...,Bistro,Hot Dog Joint,Steakhouse,Fast Food Restaurant,osm_doctor,osm_park,osm_restaurant,osm_supermarket,osm_university,osm_waste_basket
d_ref,,,,,,,,,,,,,,,,,,,,,
9,13918.551194,16702.261433,2783.710239,16702.261433,2783.710239,5567.420478,36188.233105,5567.420478,11134.840955,13918.551194,...,0.000000,0.000000,0.000000,0.00000,86295.017405,89078.727644,4.036380e+05,61241.625255,38971.943344,5.066353e+05
13,219.283608,219.283608,0.000000,219.283608,0.000000,0.000000,0.000000,0.000000,0.000000,219.283608,...,0.000000,0.000000,0.000000,0.00000,2631.403290,14692.001703,9.429195e+03,4166.388543,0.000000,1.666555e+04
1,11526.064027,40341.224093,2881.516007,8644.548020,2881.516007,8644.548020,23052.128053,2881.516007,2881.516007,8644.548020,...,2881.516007,2881.516007,8644.548020,0.00000,40341.224093,57630.320133,1.022938e+06,60511.836140,17289.096040,1.290919e+06
8,37905.187969,68229.338345,0.000000,37905.187969,7581.037594,22743.112782,98553.488720,15162.075188,7581.037594,22743.112782,...,15162.075188,7581.037594,7581.037594,0.00000,45486.225563,151620.751877,6.140640e+05,90972.451126,7581.037594,6.368072e+05
2,859.398830,1289.098244,429.699415,3007.895904,0.000000,429.699415,3007.895904,429.699415,0.000000,859.398830,...,429.699415,1289.098244,0.000000,859.39883,4726.693563,19766.173082,7.218950e+04,22774.068986,429.699415,1.564106e+05


In [38]:
print(df_vienna_features.columns)

Index(['Asian Restaurant', 'Austrian Restaurant', 'BBQ Joint', 'Bakery',
       'Breakfast Spot', 'Burger Joint', 'Café', 'Chinese Restaurant',
       'Falafel Restaurant', 'Gastropub', 'Greek Restaurant',
       'Indian Restaurant', 'Italian Restaurant', 'Japanese Restaurant',
       'Mediterranean Restaurant', 'Mexican Restaurant',
       'Middle Eastern Restaurant', 'Pizza Place', 'Restaurant',
       'Seafood Restaurant', 'Sushi Restaurant', 'Thai Restaurant',
       'Trattoria/Osteria', 'Vegetarian / Vegan Restaurant',
       'Vietnamese Restaurant', 'Bistro', 'Hot Dog Joint', 'Steakhouse',
       'Fast Food Restaurant', 'Unnamed: 0', 'osm_doctor', 'osm_park',
       'osm_restaurant', 'osm_supermarket', 'osm_university',
       'osm_waste_basket'],
      dtype='object')


## Load average flat rental prices

Its not very useful if users get recommendations for districts that fit their preferences well, but that are not affordable for them. To overcome this issue I've downloaded data about average flat rental prices from a website called www.immopreise.at and saved them to a .csv file. Since we don't expect these prices to change on regularly, we don't use automated webscraping for now.

So just read the .csv file and do some formatting.

In [28]:
df_prices = (pd
             .read_csv("rental_prices_vienna.csv", sep=";")
             .assign(d_ref = lambda x: x['Bezirk'].str.extract('(\d*)').astype(int),
                     d_name = lambda x: x['Bezirk'].str.extract('([^\d.,]+)'),
                     d_price = lambda x: x['avg_m2'].str.replace(',', '.').str.extract('([1-9.]+)').astype(float))
             .drop(columns=["Bezirk", "≤50m²", "51-80m²", "81-129m²", ">130m²", "avg_m2"])
             
            )

In [29]:
df_prices.head()

,d_ref,d_name,d_price
0,1,Innere Stadt,19.67
1,2,Leopoldstadt,16.16
2,3,Landstraße,16.16
3,4,Wieden,16.15
4,5,Margareten,15.50


## Train a XGBClassifier on our dataset 
Now we can finally train our classifier on the data we've got.

Begin by normalizing our training data.

In [43]:
scaler = preprocessing.MinMaxScaler().fit(df_vienna_features.values)

X = scaler.transform(df_vienna_features.values)

Next we need to convert our multiclass label into binary form, so **OneVsRestClassifier** can work with it.

In [44]:
lb = preprocessing.LabelBinarizer()

y = lb.fit_transform(df_vienna_features.index)

y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

It's time to create and train the classifier. We used 'binary:logistic' as an objective, since **OneVsRestClassifier** creates a single classifier for each of our districts, which then does does binary classification on how well the district fits the given construed recommendation dummy.

In [45]:
clf = OneVsRestClassifier(xgb.XGBClassifier(objective='binary:logistic'))

clf.fit(X, y)

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
          n_jobs=None)

As we don't just want the one district that fits best, but a list of districts weighted by how well they fit, we define a function that gets us this list from binarized prediction outcome.

In [46]:
def get_weighted_districts(pred):

    weighted_out = []
    
    for i in range(len(pred)-1):
        weight = pred.max()
        
        if(weight == 0):
            break
        
        label_bin = np.zeros(pred.shape)

        label_bin[pred.argmax()] = 1
    
        label_bin = np.array([label_bin])
        label = lb.inverse_transform(label_bin)
        
        

        weighted_out.append(pd.DataFrame([[label[0], weight]], columns=['d_ref', 'weight']))

        pred[pred.argmax()] = 0
        
    return pd.concat(weighted_out)
    

Let's check if we can predict a sample district from our training set correctly

In [47]:
test_from_train = clf.predict_proba(X)[1]



df_weighted_districts = (get_weighted_districts(test_from_train)
                         .merge(
                             df_prices, 
                             how='right',
                             right_on='d_ref', 
                             left_on='d_ref')
                         .set_index('d_ref')
                         .fillna(0)
                        )

df_weighted_districts.head()

,weight,d_name,d_price
d_ref,,,
13,0.126064,Hietzing,15.24
11,0.074624,Simmering,14.10
17,0.070386,Hernals,14.55
21,0.061374,Floridsdorf,15.00
22,0.058723,Donaustadt,16.00


Looks good!

## Construct a test preference

For testing our classifier, we just create a new district which contains our preferences (I just used random numbers that scale with our training data)

In [48]:
df_test_pref = pd.DataFrame(
    data=[[2000, 5000,0,0,0,0,2000,4000,9000,0,0,0,2000,4000,0,0,2000,1000,5000,0,1000,0,4000,0,0,0,0,0,0,90000,100000,0,80000,0,10000]],
    columns=df_vienna_features.columns
)

Define a maximum price we can afford (per m²)

In [49]:
max_price = 17

And run the prediction.

In [53]:
X_test = scaler.transform(df_test_pref.values)

test = clf.predict_proba(X_test)[0]



df_weighted_districts = (get_weighted_districts(test)
                         .merge(
                             df_prices, 
                             how='right',
                             right_on='d_ref', 
                             left_on='d_ref')
                         .fillna(0)
                         .reset_index()
                         .drop(columns=['index'])
                         .query('d_price <= @max_price')
                        )

df_weighted_districts['d_ref'] = df_weighted_districts['d_ref'].astype(str)


In [54]:
df_weighted_districts

,d_ref,weight,d_name,d_price
0,13,0.055909,Hietzing,15.24
2,12,0.048920,Meidling,14.37
3,8,0.048714,Josefstadt,15.59
4,17,0.047813,Hernals,14.55
5,5,0.046351,Margareten,15.50
6,6,0.045877,Mariahilf,16.42
7,9,0.040782,Alsergrund,16.27
8,11,0.040498,Simmering,14.10
9,22,0.038491,Donaustadt,16.00
10,16,0.037799,Ottakring,14.53


Now lets finally visualize our result, using a folium choropleth map.

In [55]:
nh_map = folium.Map(location=(48.2204,16.3367), zoom_start=11) 

folium.Choropleth(
    geo_data="vienna_bounds.geojson",
    data=df_weighted_districts,
    columns=['d_ref', 'weight'],
    key_on='feature.properties.ref',
    fill_color='YlOrBr',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(nh_map)
               
nh_map

## Conclusion

Getting data from OSM and foursquare is quite convenient. Especially OSM offers an incredible amount of data for free (if you bring enough time). The ability to create a .geojson file from any data brings us a bunch of new oportunities for using **folium** choropleth maps.

Getting the data from foursquare and OSM together required quite some data wrangling, but pandas does the job quite well. After data wrangling, creating the classifier was quite easy. Fortunately the **OneVsRestClassifier** class from sklearn does all the heavy lifting when building a multiclass classifier with **XGBoost**. We could have also used a normal **DecisionTree** classifier since it can do multiclass predictions out of the box, allowing to create only one classifier for the whole dataset. I didn't choose this apporach because it would have only been possible to return the one district that fits best. Also scalability would have been an issue with this approach.

The results we got from our test case seem to be quite good and they allow us to clearly visualize which district might suite the user best. Anyway, further improvements in feature scaling must be made. To create a usable product, it would be necessary to think about a way how to express the users prefernces so that they correspond with the scale of the dataset we trained our predictor on. 

I also think that if this was a real product, we would have to think about whether districts are really a good bounding area for retrieving places. If we were working with a estate agency we most likely would be able to get a list of all flats they offer. Our approach would deliver much more meaningfull if we would use the single flats adresses and a specified radius as bounding area for data retrieval. This way, the user would get very precise recommendation on which flat would fit his needs best. 

Finally I can say, that working with geo data is a lot of fun and offers many opportunities. The approach we took is quite unusual because we only have one sample per class and a large number of classes but I think it can be quite useful.

Thanks for reading.